In [1]:
#Import Depedencies
import pandas as pd
import csv

In [2]:
csvPath = "../Resources/video_game.csv"

In [3]:
#Read video_game.csv and store in Pandas DataFrame
game_df = pd.read_csv(csvPath)
game_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [4]:
#Count number of unique players
totalPlayers_dict = {}

totalPlayers = game_df["SN"].nunique()
totalPlayers_dict = {"Total Players": [totalPlayers]}

total_players_df = pd.DataFrame(totalPlayers_dict)
total_players_df

,Total Players
0,576


In [5]:
#Purchasing Analysis (Total)

uniqueItems = game_df["Item Name"].nunique()
avgPrice = round(game_df["Price"].mean(),2)
numItems = game_df["Item Name"].count()
revenue = game_df["Price"].sum()

purchasing_analytics_df = pd.DataFrame([{"Number of Unique Items": uniqueItems,
              "Average Price": avgPrice, 
               "Number of Purchases": numItems,
               "Total Revenue": revenue,}])
purchasing_analytics_df["Average Price"] = purchasing_analytics_df["Average Price"].map("${:.2f}".format)
purchasing_analytics_df["Total Revenue"] = purchasing_analytics_df["Total Revenue"].map("${:.2f}".format)
purchasing_analytics_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [6]:
#Read rows from csv into lists used in later analysis
unique_users = []
genders =[]
ages = []
items = []
unique_items_list = []
unique_items_dict = {}
unique_items = []

with open(csvPath) as csvFile:
    csvReader = csv.reader(csvFile,delimiter=",")
    header = next(csvReader,None)
    for row in csvReader:
        SN = row[1]
        gender = row[3]
        age = row[2]
        itemID = int(row[4])
        itemName = row[5]
        itemPrice = row[6]
        if SN not in unique_users:
            unique_users.append(SN)
            genders.append(gender)
            ages.append(age)
        if itemID not in unique_items_list:
            unique_items_list.append(itemID)
            unique_items_dict = {"Item ID": itemID, "Item Name":itemName}
            unique_items.append(unique_items_dict)

In [7]:
#Create unique_users_df (holds single row for each player's SN, Age)
unique_users_dict = {}
unique_users_list=[]

for i in range(len(unique_users)):
    SN = unique_users[i]
    age = int(ages[i])
    unique_users_dict = {"SN":SN , "Age":age}
    unique_users_list.append(unique_users_dict)
    
unique_user_df = pd.DataFrame(unique_users_list)

In [8]:
#Create unique_items_df (holds single row for each item's ItemID, Item Name, Price)
avg_price_dict = {}
avg_price_list = []

unique_items_df = pd.DataFrame(unique_items)
unique_items_df["Item ID"]=unique_items_df["Item ID"].astype(int)
unique_items_df.set_index("Item ID")

for item in unique_items_list:
    individual_item_df = game_df.loc[game_df["Item ID"]==item,"Price"]
    avgPrice = round(individual_item_df.mean(),2)
    avg_price_list.append(avgPrice)
    
unique_items_df["Item Price"]=avg_price_list

In [9]:
#Gender Demographics

maleCount = genders.count("Male")
malePerc = round(maleCount/totalPlayers*100,2)
femaleCount = genders.count("Female")
femalePerc = round(femaleCount/totalPlayers*100,2)
otherCount = genders.count("Other / Non-Disclosed")
otherPerc = round(otherCount/totalPlayers*100,2)
            
gender_dicts = [{"Total Count": maleCount, "Gender": "Male", "Percentage of Players":malePerc},
               {"Total Count": femaleCount, "Gender": "Female", "Percentage of Players":femalePerc},
               {"Total Count": otherCount, "Gender": "Other / Non-Disclosed", "Percentage of Players":otherPerc}]


gender_df = pd.DataFrame(gender_dicts)
gender_df = gender_df.set_index("Gender")
gender_df["Percentage of Players"] = gender_df["Percentage of Players"].map("{:.2f}%".format)
gender_df

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [10]:
#Purchasing Analysis (Gender)

#Group game_df by Gender to produce Purchase Count
gender_grouped_df = game_df.groupby("Gender")
gender_grouped_df = gender_grouped_df.count()
gender_grouped_df = gender_grouped_df.drop(["Purchase ID","Age","Item Name","Price","Item ID"],1)
gender_grouped_df = gender_grouped_df.rename(columns={"SN":"Purchase Count"})

#Locate each gender on game_df, then do calculations for avg, total, and avg total purchase

malePurchases_df = game_df.loc[game_df["Gender"]=="Male",:]
malePurchases = malePurchases_df["Item Name"].count()
avgMalePrice = round(malePurchases_df["Price"].mean(),2)
totalMalePrice = round(malePurchases_df["Price"].sum(),2)
avgPricePerMale = round(totalMalePrice/maleCount,2)

femalePurchases_df = game_df.loc[game_df["Gender"]=="Female",:]
femalePurchases = femalePurchases_df["Item Name"].count()
avgFemalePrice = round(femalePurchases_df["Price"].mean(),2)
totalFemalePrice = round(femalePurchases_df["Price"].sum(),2)
avgPricePerFemale = round(totalFemalePrice/femaleCount,2)

otherPurchases_df = game_df.loc[game_df["Gender"]=="Other / Non-Disclosed",:]
otherPurchases = otherPurchases_df["Item Name"].count()
avgOtherPrice = round(otherPurchases_df["Price"].mean(),2)
totalOtherPrice = round(otherPurchases_df["Price"].sum(),2)
avgPricePerOther = round(totalOtherPrice/otherCount,2)

#Store data into pandas DataFrame

gender_purchase_dicts = [{"Gender": "Male", "Average Purchase Price":avgMalePrice, "Total Purchase Value":totalMalePrice, "Avg Total Purchase per Person":avgPricePerMale},
                        {"Gender": "Female", "Average Purchase Price":avgFemalePrice, "Total Purchase Value":totalFemalePrice, "Avg Total Purchase per Person":avgPricePerFemale},
                        {"Gender": "Other / Non-Disclosed", "Average Purchase Price":avgOtherPrice, "Total Purchase Value":totalOtherPrice, "Avg Total Purchase per Person":avgPricePerOther}]
gender_purchase_df = pd.DataFrame(gender_purchase_dicts)
gender_purchase_df = gender_purchase_df.set_index("Gender")
grouped_gender_purchase_df = pd.merge(gender_grouped_df, gender_purchase_df, on = "Gender")

grouped_gender_purchase_df["Average Purchase Price"] = grouped_gender_purchase_df["Average Purchase Price"].map("${:.2f}".format)
grouped_gender_purchase_df["Total Purchase Value"] = grouped_gender_purchase_df["Total Purchase Value"].map("${:.2f}".format)
grouped_gender_purchase_df["Avg Total Purchase per Person"] = grouped_gender_purchase_df["Avg Total Purchase per Person"].map("${:.2f}".format)
grouped_gender_purchase_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [11]:
#Bin Age Ranges for all rows in game_df for Purchasing Analysis (Age)
bins = [0,9.9,14.9,19.9,24.9,29.9,34.9,39.9,50]
age_ranges = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

game_df["Age Ranges"] = pd.cut(game_df["Age"], bins, labels=age_ranges)

In [12]:
#Bin Age Ranges for all rows in game_df for Gender Demographics (Age)
bins = [0,9.9,14.9,19.9,24.9,29.9,34.9,39.9,50]
age_ranges = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

unique_user_df["Age Ranges"] = pd.cut(unique_user_df["Age"], bins, labels=age_ranges)

In [13]:
#Age Demographics

#Locate each age range on updated unique_user_df and perform count/percentage calculations for each range

lessThan10_df = unique_user_df.loc[unique_user_df["Age Ranges"] == "<10"]
lessThan10Count = lessThan10_df["SN"].count()
lessThan10Perc = round(lessThan10Count/totalPlayers*100,2)
tenFourteen_df = unique_user_df.loc[unique_user_df["Age Ranges"] == "10-14"]
tenFourteenCount = tenFourteen_df["SN"].count()
tenFourteenPerc = round(tenFourteenCount/totalPlayers*100,2)
fifteenNineteen_df = unique_user_df.loc[unique_user_df["Age Ranges"] == "15-19"]
fifteenNineteenCount = fifteenNineteen_df["SN"].count()
fifteenNineteenPerc = round(fifteenNineteenCount/totalPlayers*100,2)
twenty24_df = unique_user_df.loc[unique_user_df["Age Ranges"] == "20-24"]
twenty24Count = twenty24_df["SN"].count()
twenty24Perc = round(twenty24Count/totalPlayers*100,2)
twentyfive29_df = unique_user_df.loc[unique_user_df["Age Ranges"] == "25-29"]
twentyfive29Count = twentyfive29_df["SN"].count()
twentyfive29Perc = round(twentyfive29Count/totalPlayers*100,2)
thirty34_df = unique_user_df.loc[unique_user_df["Age Ranges"] == "30-34"]
thirty34Count = thirty34_df["SN"].count()
thirty34Perc = round(thirty34Count/totalPlayers*100,2)
thirtyfive39_df = unique_user_df.loc[unique_user_df["Age Ranges"] == "35-39"]
thirtyfive39Count = thirtyfive39_df["SN"].count()
thirtyfive39Perc = round(thirtyfive39Count/totalPlayers*100,2)
forty_df = unique_user_df.loc[unique_user_df["Age Ranges"] == "40+"]
fortyCount = forty_df["SN"].count()
fortyPerc = round(fortyCount/totalPlayers*100,2)

#Store data into pandas DataFrame

age_dicts = [{"Total Count": lessThan10Count, "Age Ranges": "<10", "Percentage of Players":lessThan10Perc},
            {"Total Count": tenFourteenCount, "Age Ranges": "10-14", "Percentage of Players":tenFourteenPerc},
            {"Total Count": fifteenNineteenCount, "Age Ranges": "15-19", "Percentage of Players":fifteenNineteenPerc},
            {"Total Count": twenty24Count, "Age Ranges": "20-24", "Percentage of Players":twenty24Perc},
            {"Total Count": twentyfive29Count, "Age Ranges": "25-29", "Percentage of Players":twentyfive29Perc},
            {"Total Count": thirty34Count, "Age Ranges": "30-34", "Percentage of Players":thirty34Perc},
            {"Total Count": thirtyfive39Count, "Age Ranges": "35-39", "Percentage of Players":thirtyfive39Perc},
            {"Total Count": fortyCount, "Age Ranges": "40+", "Percentage of Players":fortyPerc}]


age_df = pd.DataFrame(age_dicts)
age_df = age_df.set_index("Age Ranges")
age_df["Percentage of Players"] = age_df["Percentage of Players"].map("{:.2f}%".format)
age_df

,Total Count,Percentage of Players
Age Ranges,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [14]:
#Purchasing Analysis (Age)

#Group game_df by Age Ranges to calculate Purchase Count

grouped_age_df = game_df.groupby("Age Ranges")
grouped_age_df = grouped_age_df.count()
grouped_age_df = grouped_age_df.drop(["Purchase ID","Age","Gender","Item Name","Price","Item ID"],1)

#Locate each age range on updated game_df and perform avg, total, and avg total purchase calculations for each range

lessThan10Purchases_df = game_df.loc[game_df["Age Ranges"]=="<10",:]
avgLessThan10Price = round(lessThan10Purchases_df["Price"].mean(),2)
totalLessThan10Price = round(lessThan10Purchases_df["Price"].sum(),2)
avgPricePerLessThan10 = round(totalLessThan10Price/lessThan10Count,2)


tenFourteenPurchases_df = game_df.loc[game_df["Age Ranges"]=="10-14",:]
avgTenFourteenPrice = round(tenFourteenPurchases_df["Price"].mean(),2)
totalTenFourteenPrice = round(tenFourteenPurchases_df["Price"].sum(),2)
avgPricePerTenFourteen = round(totalTenFourteenPrice/tenFourteenCount,2)


fifteenNineteenPurchases_df = game_df.loc[game_df["Age Ranges"]=="15-19",:]
avgFifteenNineteenPrice = round(fifteenNineteenPurchases_df["Price"].mean(),2)
totalFifteenNineteenPrice = round(fifteenNineteenPurchases_df["Price"].sum(),2)
avgPricePerFifteenNineteen = round(totalFifteenNineteenPrice/fifteenNineteenCount,2)


twenty24Purchases_df = game_df.loc[game_df["Age Ranges"]=="20-24",:]
avgTwenty24Price = round(twenty24Purchases_df["Price"].mean(),2)
totalTwenty24Price = round(twenty24Purchases_df["Price"].sum(),2)
avgPricePerTwenty24 = round(totalTwenty24Price/twenty24Count,2)


twentyfive29Purchases_df = game_df.loc[game_df["Age Ranges"]=="25-29",:]
avgTwentyfive29Price = round(twentyfive29Purchases_df["Price"].mean(),2)
totalTwentyfive29Price = round(twentyfive29Purchases_df["Price"].sum(),2)
avgPricePerTwentyfive29 = round(totalTwentyfive29Price/twentyfive29Count,2)


thirty34Purchases_df = game_df.loc[game_df["Age Ranges"]=="30-34",:]
avgThirty34Price = round(thirty34Purchases_df["Price"].mean(),2)
totalThirty34Price = round(thirty34Purchases_df["Price"].sum(),2)
avgPricePerThirty34 = round(totalThirty34Price/thirty34Count,2)


thirtyfive39Purchases_df = game_df.loc[game_df["Age Ranges"]=="35-39",:]
avgThirtyfive39Price = round(thirtyfive39Purchases_df["Price"].mean(),2)
totalThirtyfive39Price = round(thirtyfive39Purchases_df["Price"].sum(),2)
avgPricePerThirtyfive39 = round(totalThirtyfive39Price/thirtyfive39Count,2)


fortyPurchases_df = game_df.loc[game_df["Age Ranges"]=="40+",:]
avgFortyPrice = round(fortyPurchases_df["Price"].mean(),2)
totalFortyPrice = round(fortyPurchases_df["Price"].sum(),2)
avgPricePerForty = round(totalFortyPrice/fortyCount,2)


age_purchase_dicts = [{"Age Ranges": "<10", "Average Purchase Price":avgLessThan10Price, "Total Purchase Value":totalLessThan10Price, "Avg Total Purchase per Person":avgPricePerLessThan10},
                     {"Age Ranges": "10-14", "Average Purchase Price":avgTenFourteenPrice, "Total Purchase Value":totalTenFourteenPrice, "Avg Total Purchase per Person":avgPricePerTenFourteen},
                     {"Age Ranges": "15-19", "Average Purchase Price":avgFifteenNineteenPrice, "Total Purchase Value":totalFifteenNineteenPrice, "Avg Total Purchase per Person":avgPricePerFifteenNineteen},
                     {"Age Ranges": "20-24", "Average Purchase Price":avgTwenty24Price, "Total Purchase Value":totalTwenty24Price, "Avg Total Purchase per Person":avgPricePerTwenty24},
                     {"Age Ranges": "25-29", "Average Purchase Price":avgTwentyfive29Price, "Total Purchase Value":totalTwentyfive29Price, "Avg Total Purchase per Person":avgPricePerTwentyfive29},
                     {"Age Ranges": "30-34", "Average Purchase Price":avgThirty34Price, "Total Purchase Value":totalThirty34Price, "Avg Total Purchase per Person":avgPricePerThirty34},
                     {"Age Ranges": "35-39", "Average Purchase Price":avgThirtyfive39Price, "Total Purchase Value":totalThirtyfive39Price, "Avg Total Purchase per Person":avgPricePerThirtyfive39},
                     {"Age Ranges": "40+", "Average Purchase Price":avgFortyPrice, "Total Purchase Value":totalFortyPrice, "Avg Total Purchase per Person":avgPricePerForty}]

age_purchase_df = pd.DataFrame(age_purchase_dicts)
age_purchase_df = age_purchase_df.set_index("Age Ranges")
age_purchase_df
grouped_age_purchase_df = pd.merge(grouped_age_df, age_purchase_df, on = "Age Ranges")      
grouped_age_purchase_df = grouped_age_purchase_df.rename(columns={"SN":"Purchase Count"})
                      
grouped_age_purchase_df["Average Purchase Price"] = grouped_age_purchase_df["Average Purchase Price"].map("${:.2f}".format)
grouped_age_purchase_df["Total Purchase Value"] = grouped_age_purchase_df["Total Purchase Value"].map("${:.2f}".format)
grouped_age_purchase_df["Avg Total Purchase per Person"] = grouped_age_purchase_df["Avg Total Purchase per Person"].map("${:.2f}".format)
grouped_age_purchase_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [15]:
#Top Spenders
spenders_list = []
spenders_dict = {}

#Group game_df by SN to calculate Purchase Count

grouped_spend_df = game_df.groupby("SN")
grouped_spend_df = grouped_spend_df.count()
grouped_spend_df = grouped_spend_df.drop(["Purchase ID","Age","Gender","Item Name","Price","Item ID"],1)

#Loop through unique_users list to perform calculations for avg and total purchase value

for i in range(len(unique_users)):
    SN = unique_users[i]
    individual_player_df = game_df.loc[game_df["SN"]==SN,:]
    totalPurchasePrice = individual_player_df["Price"].sum()
    avgPurchases = individual_player_df["Price"].mean()
    spenders_dict = {"SN":SN, "Average Purchase Price":avgPurchases, "Total Purchase Value":totalPurchasePrice}
    spenders_list.append(spenders_dict)

#Store data into pandas DataFrame

spenders_df = pd.DataFrame(spenders_list)
grouped_spend_purchase_df = pd.merge(grouped_spend_df, spenders_df, on = "SN") 
grouped_spend_purchase_df = grouped_spend_purchase_df.sort_values("Total Purchase Value",ascending = False)

grouped_spend_purchase_df = grouped_spend_purchase_df.rename(columns={"Age Ranges":"Purchase Count"})
grouped_spend_purchase_df = grouped_spend_purchase_df.set_index("SN")
grouped_spend_purchase_df["Average Purchase Price"] = grouped_spend_purchase_df["Average Purchase Price"].map("${:.2f}".format)
grouped_spend_purchase_df["Total Purchase Value"] = grouped_spend_purchase_df["Total Purchase Value"].map("${:.2f}".format)
grouped_spend_purchase_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [16]:
#Create grouped_items_df to calculate Purchase Count

grouped_items_df = game_df.groupby(["Item ID","Item Name"])
grouped_items_df = grouped_items_df.count()
grouped_items_df = grouped_items_df.drop(["Purchase ID","SN", "Age","Gender","Price"],1)

In [21]:
#Most Popular Items

#Store Item ID and Total Purchase Value into items_df
items_list = []
items_dict = {}
item_name_list = []

for i in range(len(unique_items_list)):
    itemID = int(unique_items_list[i])
    individual_item_df = game_df.loc[game_df["Item ID"]==itemID,:]
    itemPurchases = individual_item_df["Item Name"].count()
    totalItemPurchasePrice = individual_item_df["Price"].sum()
    items_dict = {"Item ID":itemID, "Total Purchase Value":totalItemPurchasePrice}
    items_list.append(items_dict)

items_df = pd.DataFrame(items_list)
items_df = items_df.set_index("Item ID")

#Merge items_df, unique_items_df, and grouped_items_df to combine results

top_items_merged_df = pd.merge(unique_items_df,items_df,on="Item ID")
top_items_merged_df = top_items_merged_df.set_index(["Item ID","Item Name"])
top_items_merged_df = pd.merge(grouped_items_df,top_items_merged_df, on = ["Item ID","Item Name"])
top_items_merged_df = top_items_merged_df.rename(columns={"Age Ranges":"Purchase Count"})

top_items_merged_df = top_items_merged_df.sort_values(["Purchase Count","Item ID"],ascending = False)

top_items_merged_df["Item Price"] = top_items_merged_df["Item Price"].map("${:.2f}".format)
top_items_merged_df["Total Purchase Value"] = top_items_merged_df["Total Purchase Value"].map("${:.2f}".format)
top_items_merged_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [20]:
#Most Profitable Items

#Sort and reformat top_items_merged_df
most_profitable_df = top_items_merged_df.sort_values("Total Purchase Value",ascending=False)
most_profitable_df["Item Price"] = most_profitable_df["Item Price"].map("${:.2f}".format)
most_profitable_df["Total Purchase Value"] = most_profitable_df["Total Purchase Value"].map("${:.2f}".format)
most_profitable_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
